The __tangle__ step in literate programming converts text in the specification language to the source language; the source language must be valid to the object language.  [`pidgin.tangle`](tangle.ipynb) transpiles __Markdown__ to valid `IPython` flavored __Python__ using the `IPython` configuration system.

Literate `pidgin` documents are reusable as __Python__ modules, documentation, and tests.

In [1]:
    import ast, IPython, re, functools, black, traitlets, doctest, textwrap, mistune, types, importnb, typing as t
    if __name__ == '__main__':  
        %reload_ext pidgin

## Explicit loaders

The tangle step idenitifies explicit statements executed to create the base state.

In [2]:
    class PidginMixin:
        extensions = '.md.ipynb',
        def code(PidginMixin, str) -> str: 
            from .specifications import transform_cell
            return importnb.loader.dedent(transform_cell.markdown_to_python(str))
        
        def visit(PidginMixin, node): 
            from .specifications import json, emojis_, yaml_, transform_ast
            """These have to be applied outer most first."""
            for visitor in reversed((
                yaml_.Yaml, json.Expand, emojis_.Emojize, transform_ast.ReturnYieldDisplay, json.Pointer, 
            )): node = visitor().visit(node)
            return node

    class PidginBase(PidginMixin, importnb.Notebook): ...
    class Pidgin(PidginBase): ...

## IPython configuration.

In [3]:
    def load_ipython_extension(shell):
        %reload_ext pidgin.specifications.transform_ast
        %reload_ext pidgin.specifications.transform_cell
        
        for i, transformer in enumerate(shell.input_transformers_cleanup):
            try:
                if transformer.initial_re.pattern[1:4] == '>>>':
                    shell.input_transformers_cleanup.pop(i)
                    break
            except: ...

The `pidgin.tangle` only operates on block level objects in the `mistune.Markdown` grammar.  The tangle step does not using and of the `mistune.Renderer` capabilities.

In [4]:
    def unload_ipython_extension(shell): ...